# ChatGPT提示工程第1课：指导原则

## 设置
加载API key和相关Python库，依赖库见requirements.txt。

In [2]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
# 在项目根目录的.env文件中填写你的OpenAI API Key
# 可以在kudaohang.com上获取测试用的key
openai.api_key  = os.getenv('OPENAI_API_KEY')

在整个课程中，我们将使用 OpenAI 的 gpt-3.5-turbo 模型和[聊天补全](https://platform.openai.com/docs/guides/chat)接口。

这个辅助函数将帮助您更轻松地使用提示并查看生成的输出。

In [3]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

提示的两个原则：
1. 使用尽可能清晰和具体的指令
2. 给AI模型留出“思考”的时间

## 原则1：使用尽可能清晰和具体的指令

### 策略一：使用分隔符来标识输入的不同部分

分隔符可以是任何东西，例如：```、"""、< >、<tag> </tag>、:。

In [4]:
text = f"""
本报北京5月3日电 （记者王珂、郑海鸥）今年“五一”假期，\
文化和旅游行业复苏势头强劲，全国假日市场平稳有序。经文化和 \
旅游部数据中心测算，全国国内旅游出游合计2.74亿人次，同比 \
增长70.83%，按可比口径恢复至2019年同期的119.09%；实现国 \
内旅游收入1480.56亿元，同比增长128.90%，按可比口径恢复至 \
2019年同期的100.66%。
"""
prompt = f"""
将用三个反引号分隔的文本总结成一句话。
```{text}```
"""
response = get_completion(prompt)
print(response)

今年“五一”假期，全国国内旅游出游人次同比增长70.83%，收入同比增长128.90%，恢复至2019年同期的119.09%和100.66%。


### 策略二：结构化输出

如JSON，HTML格式

In [19]:
prompt = f"""
​生成三本虚构中文书籍及其作者和类型的列表，并以JSON格式提供，
需包含以下键: book_id、title、author 和 genre。
"""
response = get_completion(prompt)
print(response)

1. {
   "book_id": 1,
   "title": "荒野之歌",
   "author": "张三",
   "genre": "冒险"
}

2. {
   "book_id": 2,
   "title": "幻想之城",
   "author": "李四",
   "genre": "奇幻"
}

3. {
   "book_id": 3,
   "title": "黑暗之夜",
   "author": "王五",
   "genre": "恐怖"
}


### 策略3：要求模型检查条件是否满足

In [20]:
text_1 = f"""
泡一杯茶很容易！首先，你需要把一些水烧开。在那个过程中，
拿一个杯子，并把一个茶包放进去。一旦水够热了，就把它倒在
茶包上。让茶叶浸泡一会儿。几分钟后，取出茶包。如果你喜欢，
可以加入一些糖
"""
prompt = f"""
​您将获得由三重引号分隔的文本。
如果它包含一系列说明，请按以下格式重新编写这些说明：
​
步骤1 - ...
步骤2 -…
...
步骤N -…
​
如果文本不包含一系列说明，则只需写“未提供步骤”。
​

\"\"\"{text_1}\"\"\"
"""
response = get_completion(prompt)
print("Completion for Text 1:")
print(response)

Completion for Text 1:
步骤1 - 把一些水烧开。
步骤2 - 拿一个杯子，并把一个茶包放进去。
步骤3 - 一旦水够热了，就把它倒在茶包上。
步骤4 - 让茶叶浸泡一会儿。
步骤5 - 几分钟后，取出茶包。
步骤6 - 如果你喜欢，可以加入一些糖。


In [21]:
text_2 = f"""
今天阳光明媚，鸟儿在歌唱。这是去公园散步的美好日子。
花儿绽放，树枝在微风中轻摇。人们外出，享受这美妙的天气。
有些人在野餐，有些人在玩游戏，还有些人只是在草地上放松。
这是一个完美的日子，花时间在户外欣赏大自然的美丽。
"""
prompt = f"""
将会提供一个由三引号分隔的文本。
​
如果其中包含一系列的指示，请按照以下格式重写这些指示：
步骤 1 - …
步骤 2 - …
…
步骤 N - …
如果文本不包含指示，请写“未提供步骤”。

\"\"\"{text_2}\"\"\"
"""
response = get_completion(prompt)
print("Completion for Text 2:")
print(response)

Completion for Text 2:
未提供步骤。


### 策略四：“少样本”提示

In [23]:
prompt = f"""
你的任务是用一致的风格回答问题。
​
<孩子>：教我耐心。
​
<祖父母>：最深的河谷来自一个谦虚的泉源；
最宏伟的交响乐始于一段单音；最复杂的挂毯始于一根孤独的线。
​
<孩子>：教我坚韧。
"""
response = get_completion(prompt)
print(response)

<祖父母>：坚韧不是一时的勇气，而是在困难面前不屈不挠的毅力。就像一棵树，需要经历风吹雨打，才能茁壮成长。所以，不要轻易放弃，坚持下去，你会发现自己变得更加强大。


## 原则2：给AI模型留出“思考”的时间

### 策略1：指定完成任务所需的步骤

In [36]:
text = f"""
小鸟亚历克斯梦想着能够飞得更高，更远，但由于它的翅膀太小了，\
它总是飞不高，飞不远。有一天，它看到了一只老鹰在高空翱翔，\
亚历克斯决定向老鹰请教如何飞得更高。老鹰告诉亚历克斯，\
只要拥有强大的内心和毅力，就能够飞得更高。亚历克斯听后受到了启发，\
开始每天努力练习飞行。终于有一天，它成功地飞得更高，更远，\
实现了自己的梦想
"""
# example 1
prompt_1 = f"""
执行以下操作：
​
1-用一句话总结用三个反引号分隔的以下文本。
2-将第1步的总结翻译成法语。
3-列出法语总结中的每个名字。
4-输出一个包含以下键的json对象：french_summary，num_names。
用换行符分隔您的答案。

文本：
```{text}```
"""
response = get_completion(prompt_1)
print("Completion for prompt 1:")
print(response)

Completion for prompt 1:
1- 小鸟亚历克斯通过毅力实现了飞行梦想。
2- Le petit oiseau Alex a réalisé son rêve de voler plus haut grâce à sa persévérance.
3- petit oiseau Alex
4- {"french_summary": "Le petit oiseau Alex a réalisé son rêve de voler plus haut grâce à sa persévérance.", "num_names": 1}


要求按照指定格式进行输出。

In [37]:
prompt_2 = f"""
你的任务是执行以下操作：
1-用一句话总结以下使用用<>分隔的文本。
2-将第1步的总结翻译成法语。
3-列出法语总结中的每个名字。
4-输出一个包含以下键的json对象：french_summary，num_names。

使用以下格式输出：
文本：<要进行总结的文本>
总结：<总结>
翻译：<总结的翻译>
名字：<法语总结中的名字列表>
输出JSON：<包含法语总结和num_names的json>

文本: 
<{text}>
"""
response = get_completion(prompt_2)
print("\nCompletion for prompt 2:")
print(response)


Completion for prompt 2:
总结：小鸟亚历克斯通过向老鹰请教和努力练习，成功实现了飞得更高的梦想。
翻译：Le petit oiseau Alex rêvait de voler plus haut et plus loin, mais ses ailes étaient trop petites. Après avoir demandé conseil à un aigle et s'être entraîné dur, il a finalement réussi à réaliser son rêve.
名字：Alex, l'aigle
输出JSON：{"french_summary": "Le petit oiseau Alex rêvait de voler plus haut et plus loin, mais ses ailes étaient trop petites. Après avoir demandé conseil à un aigle et s'être entraîné dur, il a finalement réussi à réaliser son rêve.", "num_names": 2}


### 策略2：在草率下结论之前，指导模型自行找出解决方案。

In [38]:
prompt = f"""
确定学生的解决方案是否正确。
​
问题：
我正在建造一个太阳能电站，需要帮助解决财务问题。
​
土地成本为每平方英尺100美元
我可以以每平方英尺250美元的价格购买太阳能电池板
我已经谈判了一份维护合同，每年的成本为固定的10万美元，以及每平方英尺10美元的额外成本
作为第一年运营的总成本是哪个平方英尺的函数。
​
学生的解决方案：
设x为平方英尺数。
成本：
1.土地成本：100x
2.太阳能电板成本：250x
3.维护成本：100,000 + 100x
总成本：100x + 250x + 100,000 + 100x = 450x + 100,000
"""
response = get_completion(prompt)
print(response)

这个学生的解决方案是正确的。他正确地列出了土地成本、太阳能电板成本和维护成本，并将它们相加得到了总成本。他还将总成本表示为x的函数，这是正确的。


注意上面的答案是错误的。
我们可以通过指导模型首先自己解决问题。
修改后的提示：

In [39]:
prompt = f"""
你的任务是确定学生的解决方案是否正确。
为了解决问题，请执行以下操作：
- 首先，解决问题并找出自己的解决方案。
- 然后将您的解决方案与学生的解决方案进行比较，
并评估学生的解决方案是否正确。
​
在你自己解决问题之前不要决定学生的解决方案是否正确。
​
​
使用以下格式：
问题：
```
问题描述
`
``
​
学生的解决方案：
```
学生的解决方案
`
``
​
实际解决方案：
```
解决方案步骤和您的解决方案
`
``
​
学生的解决方案是否与实际解决方案相同：
```
是或否
`
``
​
学生分数：
```
正确或不正确
`
``
​
​
问题：
```
我正在建造一个太阳能电站，需要帮助解决财务问题。
- 土地成本为每平方英尺100美元
- 我可以以每平方英尺250美元的价格购买太阳能电池板
- 我已经谈判了一份维护合同，每年的成本为固定的10万美元，\
以及每平方英尺10美元的额外成本
作为第一年运营的总成本是平方英尺的什么函数？
`
``
​
学生的解决方案：
```
设x为平方英尺数。
成本：
1.土地成本：100x
2.太阳能电板成本：250x
3.维护成本：100,000 + 100x
总成本：100x + 250x + 100,000 + 100x = 450x + 100,000
`
``
​
实际解决方案：
"""
response = get_completion(prompt)
print(response)

与学生的解决方案相同。
`
总成本：100x + 250x + 100,000 + 10x = 360x + 100,000
`
​
学生的解决方案是否与实际解决方案相同：
```
否
`
​
学生分数：
```
不正确
`


- [介绍](https://learn.deeplearning.ai/chatgpt-prompt-eng/lesson/1/introduction)
- [原则](https://learn.deeplearning.ai/chatgpt-prompt-eng/lesson/2/guidelines)